<h1>Conceito</h1>

Árvores de decisão são métodos de aprendizagem supervisionados não-paramétricos usados para **classificação** e **regressão**. O objetivo é criar um modelo que preveja o valor de uma variável alvo apenas aprendendo regras simples de decisão inferidas das características do conjunto de dados.

O algoritmo, em cada nó, irá considerar as diversas características presentes no conjunto de dados e, de maneira recursiva, irá testar diferentes limites de valores para essas características, tentando separar os dados em grupos homogêneos.

Considere $Q_m$ os dados que chegaram à um nó de decisão $m$, contendo $n_m$ amostras. Cada regra de decisão candidata $\theta$ consiste em uma característica $j$ e um valor limíte $t_m$, dividindo os dados e dois grupos:

- $Q_{m}^{esquerda}(\theta) = \{ (x, y) | x_j <= t_m \}$

- $Q_{m}^{direita}(\theta) = \{ (x, y) | x_j > t_m \}$

A qualidade $G(Q_m, \theta)$ da regra de decisão para esse nó é então calculada usando uma **função de impureza** ou uma **função de perca** $H()$, dependendo da tarefa à ser realizada (classificação ou regressão):

$G(Q_m, \theta) = \Large\frac{n_{m}^{esquerda}}{n_m} \normalsize H(Q_{m}^{esquerda}(\theta)) + \Large\frac{n_{m}^{direita}}{n_m} \normalsize H(Q_{m}^{direita}(\theta))$

As funções de impureza/perca disponíveis no sickit-learn são:
- Índice Gini
- Qui-Quadrado
- Ganho de informação (entropia)
- Redução na variância

Sendo utilizados em classificação o **índice Gini** e o **ganho de informação**.

<h3>Índice Gini</h3>

O índice de Gini mede quão heterogêneo os conjuntos de dados resultantes de um nó estão. Quanto mais o índice se aproxima de 1, mais heterogêneo esses conjuntos estão - ou seja, mais difícil é de se extrair informações e dividir esses grupos resultantes. Por outro lado, quanto mais o índice se aproxima de 1, mais homogêneos os conjuntos resultantes são - facilitando a subsequente divisão e extração de informações dos dados.

Para se calcular o índice Gini utiliza-se a seguinte fórmula:

$H(Q_m) = \Large\sum_{k=1}^{c} \normalsize p_{mk} (1 - p_{mk})$

onde $p_{mk}$ é a probabilidade de ocorrência da característica $j$.

<h3>Ganho de informação (entropia)</h3>

A entropia mede o nível de ordem dos conjuntos de dados resultantes. Quanto maior a entropia, mais desordenado os conjuntos de dados resultantes estão, e menor é o ganho de informação. Por outro lado, quanto menor a entropia, mais ordenado os conjuntos de dados resultantes estão, e maior é o ganho de informação.

Para se calcular a entropia utiliza-se a seguinte fórmula:

$H(Q_m) = - \Large\sum_{k=1}^{c} \normalsize p_{mk} log(p_{mk})$

onde $p_{mk}$ é a probabilidade de ocorrência da característica $j$.

<h1>Aplicação</h1>

In [1]:
#Importação das bibliotecas a serem utilizadas

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Importando a base de dados de treino/teste

dados = pd.read_csv('churn-bigml-80.csv')
dados.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [6]:
# Criando função para modificar os dados, formatando-os de forma adequada ao modelo

def modifica_dados(dados):

    # Adicionando colunas numéricas ao df novo
    dados_numericos = dados.loc[:, ~dados.columns.isin(['State', 'Area code', 'International plan', 'Voice mail plan', 'Churn'])].copy()

    # Formatando colunas de valores binários
    dic_binario = {
        'Yes': 1,
        'No': 0,
        True: 1,
        False: 0
    }

    dados_binarios = dados[['International plan', 'Voice mail plan', 'Churn']].replace(dic_binario)

    # Formatando colunas de dados com diversas classes através do método get_dummies
    dados_dummies = pd.get_dummies(dados[['Area code', 'State']])
    dados_dummies = dados_dummies.replace(dic_binario)

    # Concatenando os data frames
    dados_modificados = pd.concat([dados_numericos, dados_binarios, dados_dummies], axis = 1)

    return dados_modificados

In [7]:
# Transformado os dados de treino/teste

dados_modificados = modifica_dados(dados)

dados_modificados.head()

,Account length,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,0,0,0,0,0,0
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,...,0,0,0,0,0,0,0,0,0,0
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,...,0,0,0,0,0,0,0,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,...,0,0,0,0,0,0,0,0,0,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Dividindo inputs e outputs
x = dados_modificados.drop('Churn', axis = 1).values
y = dados_modificados['Churn'].values.ravel()

In [9]:
# Definindo random seed no numpy

SEED = 20

np.random.seed(SEED)

In [12]:
# Dividindo dados em treino e teste:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

# Instanciando o modelo
dtc = DecisionTreeClassifier()

# Realizando o treinamento
dtc.fit(x_train, y_train)

# Realizando a previsão para os dados de teste:
previsao = dtc.predict(x_test)

# Calculando o accuracy score
acuracia_dtc = accuracy_score(y_test, previsao)

print(f'Acurácia do modelo dtc treinado é: {(100*acuracia_dtc):.2f}%')

Acurácia do modelo dtc treinado é: 90.70%
